## General function to clean up data from various grants

In [1]:
import _utils
import _state_rail_plan_utils as srp_utils
import _harmonization_utils as harmonization_utils
import pandas as pd
from calitp_data_analysis.sql import to_snakecase

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
all_projects_df = harmonization_utils.add_all_projects()

/home/jovyan/data-analyses/project_list/_harmonization_utils.py:92: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_harmonization_utils.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[agency_col]
/home/jovyan/data-analyses/project_list/_harmonization_utils.py:92: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cost_columns = df.columns[df.columns.str.contains("(cost|funds)")].tolist()
/home/jovyan/data-analyses/project_list/_harmonization_utils.py:30: FutureWarning: The default value of regex will change from True to False in a future ve

In [7]:
all_projects_df.sample(10)

,project_title,lead_agency,project_category,project_description,total_project_cost,fully_funded,total_available_funds,location,county,city,notes,data_source
1634,Develop New Light Rail Corridors,None,None,None,351.10,Not fully funded,7.10,None,Santa Clara,None,None,LOST
490,9 St From Heber Ave To Rockwood Ave,None,None,Reconstruction And Widening,0.00,No project cost info,0.00,None,Imperial,City Of Calexico,None,LOST
673,Martin St From Palm Ave To 13 St,None,None,Resurface,0.00,No project cost info,0.00,None,Imperial,City Of Brawley,None,LOST
1752,Rohnert Park - Snata Rosa Avenue To Rohnert Park Exrpessway,Caltrans And Scta,Traffic Congestion Relief,This Project Would Add One Carpool Lane In Each\nDirection Through Rohnert Park And Includes The Re-Construction Of The\nWilfred Avenue Interchange And The Local Roadways In The Interchange\nArea.,40.00,Fully funded,40.00,Highway 101,Sonoma,None,None,LOST
78,South Perris Light Maintenance Facility Buildout,Metrolink,Capital,"Construction To Expand Capacity At South Perris Maintenance Facility To Accommodate 12 Consists; 10 Daily Consists, Plus Up To 2 Spares. Add Full Daily Servicing Capabilities.",153.50,No available funding info,0.00,Inland Empire Corridor,None,None,None,State Rail Plan
18,Roadway Grade Separations: Airport Way/Bnsf Stockton Subdivision,Multiple,Grade Separation,Roadway Grade Separation At Airport Way. This Is An Investment In All San Joaquins Service Between Stockton And Bakersfield.,50.00,No available funding info,0.00,Northern San Joaquin Valley Corridor\n(Valley Rail),None,None,None,State Rail Plan
1,Roadway Grade Separations: Mckinley Ave And Blackstone Ave,Sjjpa,Grade Separation,Creates Roadway Grade Separations At The Diagonal Railroad Crossing At The Corner Of Mckinely Avenue And Blackstone Avenue In Fresno. This Project Is Located On The San Joaquins Corridor (Bnsf).,80.00,No available funding info,0.00,Ca High Speed Rail Phase 1 Corridor,None,None,None,State Rail Plan
276,Verde School Road From Miller Road To 1.0 Mile East,None,None,Overlay,0.00,No project cost info,0.00,None,Imperial,City Of Imperial,None,LOST
1413,Interstate 205 Parallel Rail Freight Shuttle,None,None,Implement Rail Freight Shuttle Between The Port Of Stockton And Port Of Oakland To Divert Truck Freight Traffic\nFrom The I-205 Corridor,0.00,No project cost info,0.00,None,San Joaquin,None,None,LOST
703,Roberto Noriega St From End Of Cul De Sac To 7 St,None,None,Resurface,0.00,No project cost info,0.00,None,Imperial,City Of Brawley,None,LOST


In [6]:
all_projects_df.fully_funded.value_counts()

No project cost info         1089
No available funding info     694
Fully funded                  277
Not fully funded               65
Name: fully_funded, dtype: int64